In [46]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

In [47]:
df =  pd.read_csv('data_preprocessed.csv')

In [48]:
df['approval_flg'] = df['wo_hit_status_result_desc'].map({'Дозвон, Отказ': 0, 'Дозвон, Успешно': 1})

**Холодные продажи**

In [49]:
# датасет, где причины звонка - холодные продажи
df_cold = df[df['functional_cd'] == 'Cold']

In [50]:
df_cold['product_cd'].value_counts()

product_cd
MVNO                  664
Debit Cards           472
Mortgage Insurance    269
Casco                  18
Travel Avia             4
T-Auto                  2
Car Loan                1
Name: count, dtype: int64

Сразе можно убрать Продукты: Casco, Travel Avia, T-Auto, Car Loan. Тк данных сильно мало и выборки с подписками точно будут меньше 30

In [51]:
df_cold = df_cold[df_cold['product_cd'].isin(['MVNO', 'Debit Cards', 'Mortgage Insurance'])]

In [52]:
#оставляю только подписку Pro, premium убираем
df_cold = df_cold[df_cold['bundle_nm'] != 'Premium']
df_cold['bundle_nm'] = df_cold['bundle_nm'].fillna(0)

In [53]:
df_cold['product_cd'].value_counts()

product_cd
MVNO                  659
Debit Cards           472
Mortgage Insurance    268
Name: count, dtype: int64

Данных по каждому из продуктов нормально по количеству, далее буду проверть гипотезы на этих трех продуктах

Cold + MVNO

In [54]:
df_cold_mvno = df_cold[df_cold['product_cd'] == 'MVNO']

In [55]:
#Размеры выборок с подпиской и без неё
print(df_cold_mvno[df_cold_mvno['bundle_nm'] == 'Pro'].shape[0], df_cold_mvno[df_cold_mvno['bundle_nm'] == 0].shape[0])

187 472


In [58]:
successes_A = df_cold_mvno[(df_cold_mvno['bundle_nm'] == 0) & (df_cold_mvno['approval_flg'] == 1)].shape[0]
failures_A = df_cold_mvno[df_cold_mvno['bundle_nm'] == 0].shape[0] - successes_A
successes_B = df_cold_mvno[(df_cold_mvno['bundle_nm'] == 'Pro') & (df_cold_mvno['approval_flg'] == 1)].shape[0]
failures_B = df_cold_mvno[df_cold_mvno['bundle_nm'] == 'Pro'].shape[0] - successes_B

# Создание таблицы сопряженности
contingency_table = np.array([[successes_A, failures_A],
                               [successes_B, failures_B]])

# Применение хи-квадрат теста
chi2, p_value, _, _ = chi2_contingency(contingency_table)

# Вывод результатов
print(f"Chi-squared statistic: {chi2}")
print(f"P-value: {p_value}")

# Проверка значимости
alpha = 0.05
if p_value < alpha:
    print("Отвергаем нулевую гипотезу: различие между группами статистически значимо.")
else:
    print("Не отвергаем нулевую гипотезу: различие между группами не статистически значимо.")

Chi-squared statistic: 0.427144932186185
P-value: 0.5133932222057032
Не отвергаем нулевую гипотезу: различие между группами не статистически значимо.


Cold + Debit Cards

In [59]:
df_cold_debit = df_cold[df_cold['product_cd'] == 'Debit Cards']

In [60]:
#Размеры выборок с подпиской и без неё
print(df_cold_debit[df_cold_debit['bundle_nm'] == 'Pro'].shape[0], df_cold_debit[df_cold_debit['bundle_nm'] == 0].shape[0])

5 467


Слишком мало людей с подпиской - не смотрим

In [61]:
df_cold_mort = df_cold[df_cold['product_cd'] == 'Mortgage Insurance']

In [62]:
#Размеры выборок с подпиской и без неё
print(df_cold_mort[df_cold_mort['bundle_nm'] == 'Pro'].shape[0], df_cold_mort[df_cold_mort['bundle_nm'] == 0].shape[0])

55 213


In [63]:
successes_A = df_cold_mort[(df_cold_mort['bundle_nm'] == 0) & (df_cold_mort['approval_flg'] == 1)].shape[0]
failures_A = df_cold_mort[df_cold_mort['bundle_nm'] == 0].shape[0] - successes_A
successes_B = df_cold_mort[(df_cold_mort['bundle_nm'] == 'Pro') & (df_cold_mort['approval_flg'] == 1)].shape[0]
failures_B = df_cold_mort[df_cold_mort['bundle_nm'] == 'Pro'].shape[0] - successes_B

# Создание таблицы сопряженности
contingency_table = np.array([[successes_A, failures_A],
                               [successes_B, failures_B]])

# Применение хи-квадрат теста
chi2, p_value, _, _ = chi2_contingency(contingency_table)

# Вывод результатов
print(f"Chi-squared statistic: {chi2}")
print(f"P-value: {p_value}")

# Проверка значимости
alpha = 0.05
if p_value < alpha:
    print("Отвергаем нулевую гипотезу: различие между группами статистически значимо.")
else:
    print("Не отвергаем нулевую гипотезу: различие между группами не статистически значимо.")

Chi-squared statistic: 0.333471534850591
P-value: 0.563622037513428
Не отвергаем нулевую гипотезу: различие между группами не статистически значимо.


**Предоодбренные заявки**

In [64]:
# датасет, где причины звонка - предоодбренные заявки
df_preapprove = df[df['functional_cd'] == 'Preapprove']

In [65]:
#оставляю только подписку Pro, premium убираем
df_preapprove = df_preapprove[df_preapprove['bundle_nm'] != 'Premium']
df_preapprove['bundle_nm'] = df_preapprove['bundle_nm'].fillna(0)

In [67]:
df_preapprove['product_cd'].value_counts()

product_cd
Credit Cards    7047
Cash Loan        820
Car Loan          92
Name: count, dtype: int64

Preapprove + Credit Cards

In [68]:
df_credit_cards = df_preapprove[df_preapprove['product_cd'] == 'Credit Cards']

In [69]:
#Размеры выборок с подпиской и без неё
print(df_credit_cards[df_credit_cards['bundle_nm'] == 'Pro'].shape[0], df_credit_cards[df_credit_cards['bundle_nm'] == 0].shape[0])

1209 5838


In [70]:
successes_A = df_credit_cards[(df_credit_cards['bundle_nm'] == 0) & (df_credit_cards['approval_flg'] == 1)].shape[0]
failures_A = df_credit_cards[df_credit_cards['bundle_nm'] == 0].shape[0] - successes_A
successes_B = df_credit_cards[(df_credit_cards['bundle_nm'] == 'Pro') & (df_credit_cards['approval_flg'] == 1)].shape[0]
failures_B = df_credit_cards[df_credit_cards['bundle_nm'] == 'Pro'].shape[0] - successes_B

# Создание таблицы сопряженности
contingency_table = np.array([[successes_A, failures_A],
                               [successes_B, failures_B]])

# Применение хи-квадрат теста
chi2, p_value, _, _ = chi2_contingency(contingency_table)

# Вывод результатов
print(f"Chi-squared statistic: {chi2}")
print(f"P-value: {p_value}")

# Проверка значимости
alpha = 0.05
if p_value < alpha:
    print("Отвергаем нулевую гипотезу: различие между группами статистически значимо.")
else:
    print("Не отвергаем нулевую гипотезу: различие между группами не статистически значимо.")

Chi-squared statistic: 1.903168433102194
P-value: 0.16772409894376272
Не отвергаем нулевую гипотезу: различие между группами не статистически значимо.


Preapprove + Cash Loan

In [71]:
df_cash_loan = df_preapprove[df_preapprove['product_cd'] == 'Cash Loan']

In [72]:
#Размеры выборок с подпиской и без неё
print(df_cash_loan[df_cash_loan['bundle_nm'] == 'Pro'].shape[0], df_cash_loan[df_cash_loan['bundle_nm'] == 0].shape[0])

337 483


In [73]:
successes_A = df_cash_loan[(df_cash_loan['bundle_nm'] == 0) & (df_cash_loan['approval_flg'] == 1)].shape[0]
failures_A = df_cash_loan[df_cash_loan['bundle_nm'] == 0].shape[0] - successes_A
successes_B = df_cash_loan[(df_cash_loan['bundle_nm'] == 'Pro') & (df_cash_loan['approval_flg'] == 1)].shape[0]
failures_B = df_cash_loan[df_cash_loan['bundle_nm'] == 'Pro'].shape[0] - successes_B

# Создание таблицы сопряженности
contingency_table = np.array([[successes_A, failures_A],
                               [successes_B, failures_B]])

# Применение хи-квадрат теста
chi2, p_value, _, _ = chi2_contingency(contingency_table)

# Вывод результатов
print(f"Chi-squared statistic: {chi2}")
print(f"P-value: {p_value}")

# Проверка значимости
alpha = 0.05
if p_value < alpha:
    print("Отвергаем нулевую гипотезу: различие между группами статистически значимо.")
else:
    print("Не отвергаем нулевую гипотезу: различие между группами не статистически значимо.")

Chi-squared statistic: 1.12832995765101
P-value: 0.2881318424325282
Не отвергаем нулевую гипотезу: различие между группами не статистически значимо.


Preapprove + Car Loan

In [74]:
df_car_loan = df_preapprove[df_preapprove['product_cd'] == 'Car Loan']

In [75]:
#Размеры выборок с подпиской и без неё
print(df_car_loan[df_car_loan['bundle_nm'] == 'Pro'].shape[0], df_car_loan[df_car_loan['bundle_nm'] == 0].shape[0])

21 71


In [76]:
successes_A = df_car_loan[(df_car_loan['bundle_nm'] == 0) & (df_car_loan['approval_flg'] == 1)].shape[0]
failures_A = df_car_loan[df_car_loan['bundle_nm'] == 0].shape[0] - successes_A
successes_B = df_car_loan[(df_car_loan['bundle_nm'] == 'Pro') & (df_car_loan['approval_flg'] == 1)].shape[0]
failures_B = df_car_loan[df_car_loan['bundle_nm'] == 'Pro'].shape[0] - successes_B

# Создание таблицы сопряженности
contingency_table = np.array([[successes_A, failures_A],
                               [successes_B, failures_B]])

# Применение хи-квадрат теста
chi2, p_value, _, _ = chi2_contingency(contingency_table)

# Вывод результатов
print(f"Chi-squared statistic: {chi2}")
print(f"P-value: {p_value}")

# Проверка значимости
alpha = 0.05
if p_value < alpha:
    print("Отвергаем нулевую гипотезу: различие между группами статистически значимо.")
else:
    print("Не отвергаем нулевую гипотезу: различие между группами не статистически значимо.")

Chi-squared statistic: 0.7144130891793338
P-value: 0.39798265463641314
Не отвергаем нулевую гипотезу: различие между группами не статистически значимо.
